In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [2]:
# df_IL_2012 = pd.read_csv('IL_2012.csv')
# df_IL_2013 = pd.read_csv('IL_2013.csv')
# df_IL_2014 = pd.read_csv('IL_2014.csv')
# df_IL_2015 = pd.read_csv('IL_2015.csv')
# df_IL_2016 = pd.read_csv('IL_2016.csv')
df_IL_2017 = pd.read_csv('IL_2017.csv')
# df_IL_2012_2013 = pd.concat([df_IL_2012,df_IL_2013])
# df_IL_2012_2013.describe(include='all')
# df_IL_2012_2013.tail()
df_IL_2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17952 entries, 0 to 17951
Data columns (total 23 columns):
Partner Name               17952 non-null object
Impl License Status        17952 non-null object
Statement #                17952 non-null object
Trans. End Date            17952 non-null object
Sold Quarter               17952 non-null object
Qty                        17952 non-null int64
Rptd. Customer             17952 non-null object
Orig. Res, Customer        17789 non-null object
Res. Customer              17789 non-null object
Customer License Status    17789 non-null object
Rptd. Item                 17952 non-null object
Res. Item #                17011 non-null object
Rptd. Sale Country Name    17952 non-null object
Sale Sub-Region            17817 non-null object
Rptd. Distributor          11243 non-null object
Trans. #                   17952 non-null object
Royalty Type               17952 non-null object
Transaction Status         17952 non-null object
Claim Stag

In [3]:
df_IL_2017.rename(columns={'Partner Name':'Partner_Name','Statement #':'Statement_ID',
                           'Sold Quarter':'Sold_Quarter','Rptd. Customer':'Rptd_Customer', 
                           'Res. Customer':'Res_Customer','Rptd. Item':'Rptd_Item',
                           'Res. Item #':'Res_Item','Rptd. Distributor':'Rptd_Distributor',
                           'Royalty Type':'Royalty_Type'},inplace=True)

df_IL_2017.fillna('Blank',inplace=True)

In [4]:
df_IL_2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17952 entries, 0 to 17951
Data columns (total 23 columns):
Partner_Name               17952 non-null object
Impl License Status        17952 non-null object
Statement_ID               17952 non-null object
Trans. End Date            17952 non-null object
Sold_Quarter               17952 non-null object
Qty                        17952 non-null int64
Rptd_Customer              17952 non-null object
Orig. Res, Customer        17952 non-null object
Res_Customer               17952 non-null object
Customer License Status    17952 non-null object
Rptd_Item                  17952 non-null object
Res_Item                   17952 non-null object
Rptd. Sale Country Name    17952 non-null object
Sale Sub-Region            17952 non-null object
Rptd_Distributor           17952 non-null object
Trans. #                   17952 non-null object
Royalty_Type               17952 non-null object
Transaction Status         17952 non-null object
Claim Stag

In [5]:
def cust_status(status):
    Rptd_Customer, Res_Customer, Comments = status
        
    if Rptd_Customer == 'Blank'and Res_Customer == 'Blank':
        return 'Resolved'
    elif Rptd_Customer != 'Blank'and Res_Customer == 'Blank' and Comments == 'Blank':
        return 'Not resolved'
    elif Rptd_Customer != 'Blank'and Res_Customer == 'Blank' and Comments != 'Blank':
        return 'Resolved'
    else:
        return 'Resolved'

# def item_status(status):
#     Rptd_Item, Res_Item = status
    
#     if Rptd_Item == 'Blank' and Res_Item == 'Blank':
#         return 'Resolved'
#     elif Rptd_Item != 'Blank' and Res_Item == 'Blank':
#         return 'Not resolved'
#     else:
#         return 'Resolved'

In [6]:
df_IL_2017['Res_Cust_Status'] = df_IL_2017[['Rptd_Customer','Res_Customer','Comments']].apply(cust_status, axis=1)
# df_IL_2017['Res_Item_Status'] = df_IL_2017[['Rptd_Item','Res_Item']].apply(item_status, axis=1)

In [7]:
df_IL_2017.head(2)

,Partner_Name,Impl License Status,Statement_ID,Trans. End Date,Sold_Quarter,Qty,Rptd_Customer,"Orig. Res, Customer",Res_Customer,Customer License Status,...,Rptd_Distributor,Trans. #,Royalty_Type,Transaction Status,Claim Stage,Eval/Sample Distributed,Comments,SF Entity Id,SAP Entity ID,Res_Cust_Status
0,ALI CORPORATION,Active,IL-2018005831,31-Dec-17,2017-Q4,10800,ABOX42 GMBH,ABOX42 GMBH,ABOX42 GMBH,Active,...,YOSUN,POS-521518,IL Report,Closed,Blank,Blank,Blank,2030,105427,Resolved
1,ALI CORPORATION,Active,IL-2018005831,31-Dec-17,2017-Q4,225,ABOX42 GMBH,ABOX42 GMBH,ABOX42 GMBH,Active,...,YOSUN,POS-521519,IL Report,Closed,Blank,Blank,Blank,2030,105427,Resolved


In [12]:
df_IL_2017Agg = df_IL_2017.groupby(['Partner_Name','Res_Cust_Status'])['Qty'].agg(['sum','count'])
pv = pd.pivot_table(df_IL_2017, index=['Partner_Name'], columns=["Res_Cust_Status"], 
                    values=['Qty'], aggfunc=sum, fill_value=0)
pv

Qty           
Res_Cust_Status                                    Not resolved   Resolved
Partner_Name                                                              
ALI CORPORATION                                               0    4335453
ALLWINNER TECHNOLOGY CO LTD                                   0      15280
AMLOGIC INC                                                   0   23957661
ANALOG DEVICES INC                                           20   12824465
ASAHI KASEI MICROSYSTEMS CO LTD                               0     199900
AVAILINK INC                                                  0     403320
AVNERA CORPORATION                                            0     674642
BROADCOM CORPORATION                                     946784  104163516
C-MEDIA ELECTRONICS INC                                       0     107050
CHANGSHA XIANGJIA INVESTMENT CO LTD                           0        380
CIRRUS LOGIC INC                                             50    2376367
CONEXANT SYSTEMS INC                                          0     216000
ESS TECHNOLOGY INC                                            0    3690871
FREESCALE SEMICONDUCTOR INC                                   0     986958
FUZHOU ROCKCHIP ELECTRONICS CO LTD                            0     375885
GIGPEAK INC                                                   0       5428
HANGZHOU NATIONALCHIP SCIENCE AND TECHNOLOGY CO...            0     468409
HISILICON TECHNOLOGIES CO LTD                                20   55344342
INTEGRATED DEVICE TECHNOLOGY INC                              0       3140
INTEL CORPORATION                                             0    1144852
INTERSIL CORPORATION                                          0     443582
MARVELL INTERNATIONAL LTD                                     0    1918539
MAXLINEAR INC                                                 0     365120
MCS LOGIC INC                                                 0     629661
MEDIATEK INC                                                  0   51684605
MONTAGE TECHNOLOGY GROUP LTD                                  0     233063
MSTAR SEMICONDUCTOR INC                                     640   59971785
NEW JAPAN RADIO CO LTD                                        0       4000
NOVATEK MICROELECTRONICS CORP                                 0   23527649
ON SEMICONDUCTOR                                              0    3605614
OPTEK DIGITAL TECHNOLOGY CO LTD                               0      11200
PIXTREE INC                                                   0       3166
PRINCETON TECHNOLOGY CORPORATION                              0      86776
Panasonic Corporation                                         0    8274771
Pioneer Corporation                                           0       7755
QUALCOMM TECHNOLOGIES INTERNATIONAL LTD                  503965    1839900
RDA TECHNOLOGIES LIMITED                                      0    3209440
REALTEK SEMICONDUCTOR CORP                                    0    9201360
RENESAS ELECTRONICS CORPORATION                          235307    3751228
SANECHIPS TECHNOLOGY CO LTD                                   0    3104112
STMICROELECTRONICS INTERNATIONAL NV                     -228176   13554427
SUNPLUS TECHNOLOGY CO LTD                                     0    7418274
SYNAPTICS INCORPORATED                                        0     769451
Samsung Electronics Co., Ltd.                                 0     106868
Socionext Inc.                                                0   10670398
Sony Corporation                                              0    3473118
TELECHIPS INC                                                 0    1269002
TEXAS INSTRUMENTS INCORPORATED                                0    2671931
Techno Mathematical Co., Ltd.                                 0     757173
Toshiba Corporation                                           0     164810
VIXS SYSTEMS INC                                              0      45512

In [9]:
# df_IL_2017Agg
# df_IL_2017.to_csv('IL_2017_results.csv')
# df_IL_2017Agg.to_csv('IL_2017_aggregated.csv')

In [10]:
# import ipywidgets as widgets
# from IPython.display import display
# from IPython.html.widgets import interactive

# items = ['All']+sorted(df_IL_2017['Partner_Name'].unique().tolist())

# def view(Partner=''):
#     if Partner== 'All':
#         return df_IL_2017
#     else:
#         return df_IL_2017[df_IL_2017['Partner_Name']==Partner]
    
# w = widgets.Select(options=items)
# interactive(view, Partner=w)
